In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; 

## Helper function to extract the name of the author into a sepreate colum

In [122]:
def authors2author(name):
    tree = ET.parse(f'./data-cleaned/{name}.xml')    
    root = tree.getroot()
    for child in root:
        c = ET.Element("author")
        Names=child.findall('authors')[0].findall("author")[0].findall("name")
        text=""
        if len(Names)==1:
            text=Names[0].text
        else:
            for Name in Names:
                text+=Name.text+"/"
        c.text = text
        child.insert(2, c)
        child.remove(child.findall('authors')[0])
    tree=ET.ElementTree(root)
    with open(f'{name}M.xml', 'wb') as f:
        tree.write(f)

In [123]:
authors2author("kindle")
authors2author('recommendation')

# First recommendation and kindle data sets

In [124]:
df_kindle = pd.read_xml("kindleM.xml")

In [128]:
df_kindle

,id,title,author,rating,pages,price,language,publisher
0,kindle_1,How to be a Bartender: Get a Bartending Job Wi...,Dave Allred,4.0,171.0,9.0,English,None
1,kindle_2,Raise the Bar: An Action-Based Method for Maxi...,Jon Taffer,4.0,258.0,2.0,English,Amazon Publishing
2,kindle_3,The Professional Bartender's Handbook: A Recip...,Valerie Mellema,4.0,288.0,9.0,English,Atlantic Publishing Group Inc.; illustrated ed...
3,kindle_4,How to Double Your Tips,Jerry Minchey,4.0,36.0,2.0,English,Stony River Media
4,kindle_5,The Bartender's Bible: 1001 Mixed Drinks,Gary Regan,3.0,400.0,5.0,English,HarperCollins e-books
...,...,...,...,...,...,...,...,...
49179,kindle_49180,"Famous Gamblers, Poker History, and Texas Stories",Johnny Hughes,4.0,256.0,6.0,English,iUniverse
49180,kindle_49181,The Madness of March: Bonding and Betting with...,Alan Jay Zaremba,3.0,243.0,9.0,English,University of Nebraska Press
49181,kindle_49182,The Football Betting Bible,Stephen Ward,4.0,70.0,6.0,English,None
49182,kindle_49183,Winning on Betfair For Dummies,Alex Gowar,NaN,210.0,10.0,English,For Dummies; 2nd Revised edition edition


In [126]:
df_recommendation = pd.read_xml("recommendationM.xml")
df_recommendation

,id,title,author,year,publisher
0,recommendation_1,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,recommendation_2,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,recommendation_3,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,recommendation_4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,recommendation_5,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271313,recommendation_271314,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271314,recommendation_271315,From One to One Hundred,Teri Sloat,1991,Dutton Books
271315,recommendation_271316,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271316,recommendation_271317,Republic (World's Classics),Plato,1996,Oxford University Press


In [51]:
Gold_df=pd.merge(df_recommendation, df_kindle,  left_on=['title','author'], right_on = ['title','author']) 
Gold_df=Gold_df.drop(['title','author','year','rating','pages','price','language'],1)
Gold_df["gold"] =  np.where(Gold_df['publisher_x'] == Gold_df['publisher_y'],True,False)
Gold_df

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,id_x,publisher_x,id_y,publisher_y,gold
0,recommendation_19,Dell,kindle_97,Dell,True
1,recommendation_950,Doubleday Books,kindle_97,Dell,False
2,recommendation_4316,Trafalgar Square,kindle_97,Dell,False
3,recommendation_130480,Bantam Books,kindle_97,Dell,False
4,recommendation_178745,Random House Audio,kindle_97,Dell,False
...,...,...,...,...,...
4216,recommendation_270245,Benbella Books,kindle_47716,BenBella Books,False
4217,recommendation_270289,Perseus Publishing,kindle_21344,Basic Books; Second Edition edition,False
4218,recommendation_270699,Pomegranate,kindle_21910,None,False
4219,recommendation_270720,Berrett-Koehler Publishers,kindle_575,Berrett-Koehler Publishers; Second Edition edi...,False


In [52]:
Gold_T=Gold_df.drop(Gold_df[Gold_df['gold'] == False].index)
Gold_T

,id_x,publisher_x,id_y,publisher_y,gold
0,recommendation_19,Dell,kindle_97,Dell,True
51,recommendation_65,Ballantine Books,kindle_13763,Ballantine Books,True
53,recommendation_74343,Ballantine Books,kindle_13763,Ballantine Books,True
154,recommendation_514,Atlantic Monthly Press,kindle_28945,Atlantic Monthly Press,True
162,recommendation_648,Scribner,kindle_18205,Scribner,True
...,...,...,...,...,...
4143,recommendation_241369,See Sharp Press,kindle_22023,See Sharp Press,True
4147,recommendation_241896,Hyperion,kindle_8426,Hyperion,True
4152,recommendation_243493,HarperBusiness,kindle_7708,HarperBusiness,True
4161,recommendation_249702,Penguin Books,kindle_34571,Penguin Books,True


### This will be part of the True part of the GS

In [57]:
Gold_H=Gold_df.drop(Gold_df[Gold_df['gold'] == True].index)
Gold_H=Gold_H.sort_values(by=['publisher_y'], ascending=False)
Gold_H.reset_index( inplace=True)
Gold_H.drop('index',1,inplace=True)
Gold_H

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,id_x,publisher_x,id_y,publisher_y,gold
0,recommendation_37262,Push,kindle_26687,namelos,False
1,recommendation_116505,Critics Choice Paperbacks,kindle_33088,iUniverse,False
2,recommendation_197924,Kensington Pub Corp (Mm),kindle_23553,ePublishing Works!,False
3,recommendation_235742,Pinnacle Books,kindle_32947,ePublishing Works!,False
4,recommendation_65224,HarperTorch,kindle_4346,ePublishing Works!,False
...,...,...,...,...,...
3922,recommendation_251496,Bantam Books,kindle_29019,None,False
3923,recommendation_258325,Dover Publications,kindle_10136,None,False
3924,recommendation_258326,Dover Publications,kindle_25924,None,False
3925,recommendation_260517,Fawcett Books,kindle_34980,None,False


In [73]:
Gold_H.loc[1:380].to_csv('recommendation_kindle_H.csv')

# After that some of the data were evaluated by hand and then read again

In [78]:
Gold_H_D = pd.read_csv("recommendation_kindle_H.csv")
Gold_H_D

,Unnamed: 0,id_x,publisher_x,id_y,publisher_y,gold
0,1,recommendation_116505,Critics Choice Paperbacks,kindle_33088,iUniverse,False
1,2,recommendation_197924,Kensington Pub Corp (Mm),kindle_23553,ePublishing Works!,False
2,3,recommendation_235742,Pinnacle Books,kindle_32947,ePublishing Works!,False
3,4,recommendation_65224,HarperTorch,kindle_4346,ePublishing Works!,False
4,5,recommendation_9970,Ivy Books,kindle_3150,eKensington,False
...,...,...,...,...,...,...
279,376,recommendation_35794,Tor Books,kindle_24643,Tor Books; Reprint edition,True
280,377,recommendation_128972,Putnam Pub Group,kindle_35184,Tor Books; Reprint edition,False
281,378,recommendation_6059,Berkley Publishing Group,kindle_35184,Tor Books; Reprint edition,False
282,379,recommendation_164863,Tor Books,kindle_39013,Tor Books; First Edition edition,True


In [79]:
Gold_H_D.drop('Unnamed: 0',1,inplace=True)

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [84]:
gs_recommendation_kindle=Gold_T.drop(['publisher_x','publisher_y'],1).append(Gold_H_D.drop(Gold_H_D[Gold_H_D['gold'] == False].index).drop(['publisher_x','publisher_y'],1)).append(Gold_H_D.drop(Gold_H_D[Gold_H_D['gold'] == True].index).drop(['publisher_x','publisher_y'],1))
gs_recommendation_kindle.reset_index( inplace=True)
gs_recommendation_kindle.drop('index',1,inplace=True)
gs_recommendation_kindle

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,id_x,id_y,gold
0,recommendation_19,kindle_97,True
1,recommendation_65,kindle_13763,True
2,recommendation_74343,kindle_13763,True
3,recommendation_514,kindle_28945,True
4,recommendation_648,kindle_18205,True
...,...,...,...
573,recommendation_11982,kindle_30046,False
574,recommendation_213132,kindle_30046,False
575,recommendation_142423,kindle_38989,False
576,recommendation_128972,kindle_35184,False


# Then we have our gold standard

In [85]:
gs_recommendation_kindle.to_csv('./gs/gs_recommendation_kindle.csv')

# Second kindle and goodreads data sets

In [129]:
authors2author("goodreads")

In [160]:
df_goodreads = pd.read_xml("goodreadsM.xml")
df_goodreads

,id,title,author,rating,pages,genres
0,goodreads_1,The Hunger Games,Suzanne Collins,4,374.0,NaN
1,goodreads_2,Harry Potter and the Order of the Phoenix,J.K. Rowling/Mary GrandPré/,4,870.0,NaN
2,goodreads_3,To Kill a Mockingbird,Harper Lee,4,324.0,NaN
3,goodreads_4,Pride and Prejudice,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,4,279.0,NaN
4,goodreads_5,Twilight,Stephenie Meyer,3,498.0,NaN
...,...,...,...,...,...,...
54295,goodreads_54296,Taking the Field: A Fan's Quest to Run the Tea...,Howard Megdal,3,256.0,NaN
54296,goodreads_54297,"The Baseball Talmud: Koufax, Greenberg, and th...",Howard Megdal,3,256.0,NaN
54297,goodreads_54298,"Wilpon's Folly - The Story of a Man, His Fortu...",Howard Megdal,3,NaN,NaN
54298,goodreads_54299,He Wanted the Moon: The Madness and Medical Ge...,Mimi Baird/Eve Claxton/,3,272.0,NaN


In [131]:
df_goodreads_2=df_goodreads.drop(['genres','rating'],1)
df_kindle_2=df_kindle.drop(['price','language','rating','publisher'],1)

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [164]:
Gold_df=pd.merge(df_goodreads_2, df_kindle_2,  left_on=['title','pages'], right_on = ['title','pages']) 
Gold_df["gold"] =  np.where(Gold_df['author_x'] == Gold_df['author_y'],True,False)
Gold_df.drop(["title","pages"],1,inplace=True)
Gold_df

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,id_x,author_x,id_y,author_y,gold
0,goodreads_34,John Steinbeck,kindle_3245,John Steinbeck,True
1,goodreads_70,Sylvia Plath/Reinhard Kaiser/,kindle_3612,Sylvia Plath,False
2,goodreads_31103,Sylvia Plath,kindle_3612,Sylvia Plath,True
3,goodreads_126,Umberto Eco/William Weaver/Seán Barrett/,kindle_9696,Umberto Eco,False
4,goodreads_335,Ann Patchett,kindle_1083,Ann Patchett,True
...,...,...,...,...,...
492,goodreads_53653,Jodi Lynn Anderson,kindle_45411,Jodi Lynn Anderson,True
493,goodreads_53671,Derek Sanderson/Kevin Shea/,kindle_22268,Derek Sanderson,False
494,goodreads_53733,Neil Young,kindle_757,Neil Young,True
495,goodreads_53739,Kenneth Grahame,kindle_25913,Kenneth Grahame,True


In [140]:
Gold_T=Gold_df.drop(Gold_df[Gold_df['gold'] == False].index)
Gold_T

,id_x,author_x,id_y,author_y,gold
0,goodreads_34,John Steinbeck,kindle_3245,John Steinbeck,True
2,goodreads_31103,Sylvia Plath,kindle_3612,Sylvia Plath,True
4,goodreads_335,Ann Patchett,kindle_1083,Ann Patchett,True
5,goodreads_344,Marie Lu,kindle_2493,Marie Lu,True
6,goodreads_12528,Marie Lu,kindle_2493,Marie Lu,True
...,...,...,...,...,...
490,goodreads_53398,Mark Z. Danielewski,kindle_41250,Mark Z. Danielewski,True
492,goodreads_53653,Jodi Lynn Anderson,kindle_45411,Jodi Lynn Anderson,True
494,goodreads_53733,Neil Young,kindle_757,Neil Young,True
495,goodreads_53739,Kenneth Grahame,kindle_25913,Kenneth Grahame,True


In [145]:
Gold_H=Gold_df.drop(Gold_df[Gold_df['gold'] == True].index)
Gold_H=Gold_H.sort_values(by=['author_y'], ascending=False)
Gold_H.reset_index( inplace=True)
Gold_H.drop('index',1,inplace=True)
Gold_H

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,id_x,author_x,id_y,author_y,gold
0,goodreads_33803,Zilpha Keatley Snyder/Alton Raible/,kindle_27124,Zilpha Keatley Snyder,False
1,goodreads_37474,Yotam Ottolenghi/Sami Tamimi/Richard Learoyd/,kindle_4164,Yotam Ottolenghi,False
2,goodreads_34027,Yōko Ogawa/Stephen Snyder/,kindle_46514,Yoko Ogawa,False
3,goodreads_26011,Nicola Haken,kindle_22733,William Cope Moyers,False
4,goodreads_35785,Whittaker Chambers/William F. Buckley Jr./Robe...,kindle_11818,Whittaker Chambers,False
...,...,...,...,...,...
150,goodreads_16463,Angelo Dirks/Leland Dirks/,kindle_3538,Angelo Dirks,False
151,goodreads_50176,Angela Ackerman/Becca Puglisi/,kindle_4931,Angela Ackerman,False
152,goodreads_2317,Amie Kaufman/Meagan Spooner/,kindle_21065,Amie Kaufman,False
153,goodreads_34625,Alison G. Bailey,kindle_27442,Alison Bailey,False


In [148]:
Gold_H.to_csv('goodreads_kindle_H.csv')

# After that some of the data were evaluated by hand and then read again

In [156]:
Gold_H_D = pd.read_csv('goodreads_kindle_H.csv')
Gold_H_D.drop('Unnamed: 0',1,inplace=True)
gs_goodreads_kindle=Gold_T.drop(['author_x','author_y'],1).append(Gold_H_D.drop(Gold_H_D[Gold_H_D['gold'] == False].index).drop(['author_x','author_y'],1)).append(Gold_H_D.drop(Gold_H_D[Gold_H_D['gold'] == True].index).drop(['author_x','author_y'],1))
gs_goodreads_kindle.reset_index( inplace=True)
gs_goodreads_kindle.drop('index',1,inplace=True)
gs_goodreads_kindle

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


,id_x,id_y,gold
0,goodreads_34,kindle_3245,True
1,goodreads_31103,kindle_3612,True
2,goodreads_335,kindle_1083,True
3,goodreads_344,kindle_2493,True
4,goodreads_12528,kindle_2493,True
...,...,...,...
492,goodreads_19138,kindle_19046,False
493,goodreads_22985,kindle_7462,False
494,goodreads_33014,kindle_1067,False
495,goodreads_15558,kindle_26077,False


### We noticed most of the data are matched, so we will repeate the process with some tweeks

### This function to create additional columns to help out filtering the data

In [221]:
def authors2authorTry(name):
    tree = ET.parse(name+".xml")    
    root = tree.getroot()
    for child in root:
        Names=child.findall('authors')[0].findall("author")[0].findall("name")
        y = ET.Element("author")
        y.text=Names[0].text
        child.insert(1, y)
        if len(Names)>1:
            k = ET.Element("Mul")
            k.text='TRUE'
            child.insert(3, k)
        text=""
        if len(Names)==1:
            text=Names[0].text
        else:
            for Name in Names:
                text+=Name.text+"/"
        c = ET.Element("Allauthors")
        c.text = text
        child.insert(1, c)
        child.remove(child.findall('authors')[0])
    tree=ET.ElementTree(root)
    with open(f'{name}MT.xml', 'wb') as f:
        tree.write(f)


In [222]:
authors2authorTry("goodreads")
df_g = pd.read_xml("goodreadsMT.xml")
df_g

,id,Allauthors,author,title,rating,pages,genres,Mul
0,goodreads_1,Suzanne Collins,Suzanne Collins,The Hunger Games,4,374.0,NaN,None
1,goodreads_2,J.K. Rowling/Mary GrandPré/,J.K. Rowling,Harry Potter and the Order of the Phoenix,4,870.0,NaN,TRUE
2,goodreads_3,Harper Lee,Harper Lee,To Kill a Mockingbird,4,324.0,NaN,None
3,goodreads_4,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,Jane Austen,Pride and Prejudice,4,279.0,NaN,TRUE
4,goodreads_5,Stephenie Meyer,Stephenie Meyer,Twilight,3,498.0,NaN,None
...,...,...,...,...,...,...,...,...
54295,goodreads_54296,Howard Megdal,Howard Megdal,Taking the Field: A Fan's Quest to Run the Tea...,3,256.0,NaN,None
54296,goodreads_54297,Howard Megdal,Howard Megdal,"The Baseball Talmud: Koufax, Greenberg, and th...",3,256.0,NaN,None
54297,goodreads_54298,Howard Megdal,Howard Megdal,"Wilpon's Folly - The Story of a Man, His Fortu...",3,NaN,NaN,None
54298,goodreads_54299,Mimi Baird/Eve Claxton/,Mimi Baird,He Wanted the Moon: The Madness and Medical Ge...,3,272.0,NaN,TRUE


In [223]:
df_g=df_g.drop(['genres','rating'],1)
df_k=df_kindle.drop(['price','language','rating','publisher'],1)

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [234]:
G_df=pd.merge(df_g, df_k,  left_on=['title'], right_on = ['title']) 
G_df["gold"] =  np.where(G_df['pages_x'] == G_df['pages_y'],True,False)
G_df["author_gold"] =  np.where(G_df['author_x'] == G_df['author_y'],True,False)
# G_df.drop(["title","author"],1,inplace=True)
G_df

,id_x,Allauthors,author_x,title,pages_x,Mul,id_y,author_y,pages_y,gold,author_gold
0,goodreads_1,Suzanne Collins,Suzanne Collins,The Hunger Games,374.0,None,kindle_5559,Suzanne Collins,387.0,False,True
1,goodreads_14937,Suzanne Collins,Suzanne Collins,The Hunger Games,454.0,None,kindle_5559,Suzanne Collins,387.0,False,True
2,goodreads_21678,Suzanne Collins,Suzanne Collins,The Hunger Games,374.0,None,kindle_5559,Suzanne Collins,387.0,False,True
3,goodreads_22255,Suzanne Collins,Suzanne Collins,The Hunger Games,436.0,None,kindle_5559,Suzanne Collins,387.0,False,True
4,goodreads_4,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,Jane Austen,Pride and Prejudice,279.0,TRUE,kindle_9845,Jane Austen,258.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...
5459,goodreads_54040,"James Wesley, Rawles","James Wesley, Rawles",How to Survive the End of the World as We Know...,336.0,None,kindle_19680,JamesWesley Rawles,340.0,False,False
5460,goodreads_54043,Richard Price,Richard Price,The Wanderers,NaN,None,kindle_21113,Richard Price,260.0,False,True
5461,goodreads_54051,Kazuo Ishiguro,Kazuo Ishiguro,When We Were Orphans,336.0,None,kindle_9715,Kazuo Ishiguro,352.0,False,True
5462,goodreads_54134,Nadine Gordimer,Nadine Gordimer,The Conservationist,267.0,None,kindle_47622,Nadine Gordimer,271.0,False,True


In [235]:
G_df=G_df.drop(G_df[G_df['author_gold'] == True].index)
G_df=G_df.drop(G_df[G_df['gold'] == True].index)
G_df=G_df.drop(G_df[G_df['Mul'] != "TRUE"].index)
G_df=G_df.drop(['author_x','title','pages_x','Mul','pages_y','author_gold'],1)
G_df.reset_index( inplace=True)
G_df.drop('index',1,inplace=True)
G_df

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,id_x,Allauthors,id_y,author_y,gold
0,goodreads_34803,Elie Wiesel/Marion Wiesel/,kindle_37949,Anya Bast,False
1,goodreads_14,Emily Brontë/Richard J. Dunn/David Timson/Char...,kindle_25002,Emily Bronte,False
2,goodreads_14,Emily Brontë/Richard J. Dunn/David Timson/Char...,kindle_25220,Emily Bronte,False
3,goodreads_15034,Emily Brontë/Pauline Nestor/Lucasta Miller/,kindle_25002,Emily Bronte,False
4,goodreads_15034,Emily Brontë/Pauline Nestor/Lucasta Miller/,kindle_25220,Emily Bronte,False
...,...,...,...,...,...
411,goodreads_51673,Marcus Annaeus Lucanus/Matthew Fox/Ethan Adams/,kindle_25990,Mark Millar,False
412,goodreads_52118,J.D. Robb/Nora Roberts/,kindle_13514,J. D. Robb,False
413,goodreads_52570,J.D. Robb/Nora Roberts/,kindle_4586,J. D. Robb,False
414,goodreads_53094,Antonio J. Méndez/Matt Baglio/Dylan Baker/,kindle_22008,Antonio Mendez,False


### We merged based on title and choose the books with different pages and the first author is different, Now we need to make sure by hand that these entites are really different

In [236]:
G_df.loc[1:250].to_csv('goodreads_kindle_HT.csv')

In [241]:
after_H = pd.read_csv("goodreads_kindle_HT.csv")
after_H.drop('Unnamed: 0',1,inplace=True)
after_H=after_H.drop(after_H[after_H['gold'] == True].index)
after_H

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,id_x,Allauthors,id_y,author_y,gold
40,goodreads_18600,Joseph Conrad/Owen Knowles/Robert Hampson/,kindle_25018,Matt Kish,False
41,goodreads_27951,Joseph Conrad/Chinua Achebe/,kindle_25018,Matt Kish,False
44,goodreads_37011,Malvina G. Vogel/Lorna Tomai/,kindle_25913,Kenneth Grahame,False
46,goodreads_296,John Milton/John Leonard/,kindle_15580,Robert Gates Sr.,False
47,goodreads_24147,John Milton/David Hawkes/,kindle_15580,Robert Gates Sr.,False
...,...,...,...,...,...
244,goodreads_26643,Pat Graversen/Richard Newton/,kindle_47337,Anne Stuart,False
245,goodreads_44254,Becca Fitzpatrick/Mariusz Gądek/,kindle_47337,Anne Stuart,False
246,goodreads_10119,George R.R. Martin/Walter Jon Williams/Melinda...,kindle_47322,Simone Elkeles,False
247,goodreads_7972,Gregory Maguire/Douglas Smith/,kindle_5327,Jacqueline Davies,False


### We need to revise these data to be sure .We now will examine the attruibites by hand

In [242]:
after_H.to_csv('goodreads_kindle_H_revise.csv')

In [265]:
after_R = pd.read_csv("goodreads_kindle_H_revise.csv")
after_R.drop('Unnamed: 0',1,inplace=True)
after_R.drop(['Allauthors','author_y'],1,inplace=True)
# after_R=after_R.drop(after_R[after_R['gold'] == True].index)
after_R

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,id_x,id_y,gold
0,goodreads_18600,kindle_25018,False
1,goodreads_27951,kindle_25018,False
2,goodreads_37011,kindle_25913,False
3,goodreads_296,kindle_15580,False
4,goodreads_24147,kindle_15580,False
...,...,...,...
169,goodreads_26643,kindle_47337,False
170,goodreads_44254,kindle_47337,False
171,goodreads_10119,kindle_47322,False
172,goodreads_7972,kindle_5327,False


In [266]:
gs_goodreads_kindle

,id_x,id_y,gold
0,goodreads_34,kindle_3245,True
1,goodreads_31103,kindle_3612,True
2,goodreads_335,kindle_1083,True
3,goodreads_344,kindle_2493,True
4,goodreads_12528,kindle_2493,True
...,...,...,...
492,goodreads_19138,kindle_19046,False
493,goodreads_22985,kindle_7462,False
494,goodreads_33014,kindle_1067,False
495,goodreads_15558,kindle_26077,False


In [269]:
gs_goodreads_kindle_final=gs_goodreads_kindle.append(after_R)
gs_goodreads_kindle_final.reset_index( inplace=True)
gs_goodreads_kindle_final.drop('index',1,inplace=True)
gs_goodreads_kindle_final

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,id_x,id_y,gold
0,goodreads_34,kindle_3245,True
1,goodreads_31103,kindle_3612,True
2,goodreads_335,kindle_1083,True
3,goodreads_344,kindle_2493,True
4,goodreads_12528,kindle_2493,True
...,...,...,...
666,goodreads_26643,kindle_47337,False
667,goodreads_44254,kindle_47337,False
668,goodreads_10119,kindle_47322,False
669,goodreads_7972,kindle_5327,False


In [270]:
gs_goodreads_kindle_final.drop(gs_goodreads_kindle_final[gs_goodreads_kindle_final['gold'] == True].index)


,id_x,id_y,gold
485,goodreads_26011,kindle_22733,False
486,goodreads_46500,kindle_22696,False
487,goodreads_25768,kindle_10755,False
488,goodreads_49123,kindle_10664,False
489,goodreads_19581,kindle_11130,False
...,...,...,...
666,goodreads_26643,kindle_47337,False
667,goodreads_44254,kindle_47337,False
668,goodreads_10119,kindle_47322,False
669,goodreads_7972,kindle_5327,False


### Finally We have our gold standard

In [271]:
gs_goodreads_kindle_final.to_csv('./gs/gs_goodreads_kindle.csv')

# Third: goodreads and recommendation datasets

In [272]:
df_recommendation = pd.read_xml("recommendationM.xml")
df_recommendation

,id,title,author,year,publisher
0,recommendation_1,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,recommendation_2,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,recommendation_3,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,recommendation_4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,recommendation_5,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271313,recommendation_271314,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271314,recommendation_271315,From One to One Hundred,Teri Sloat,1991,Dutton Books
271315,recommendation_271316,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271316,recommendation_271317,Republic (World's Classics),Plato,1996,Oxford University Press


In [276]:
df_goodreadsMT = pd.read_xml("goodreadsMT.xml")
df_goodreadsMT

,id,Allauthors,author,title,rating,pages,genres,Mul
0,goodreads_1,Suzanne Collins,Suzanne Collins,The Hunger Games,4,374.0,NaN,None
1,goodreads_2,J.K. Rowling/Mary GrandPré/,J.K. Rowling,Harry Potter and the Order of the Phoenix,4,870.0,NaN,TRUE
2,goodreads_3,Harper Lee,Harper Lee,To Kill a Mockingbird,4,324.0,NaN,None
3,goodreads_4,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,Jane Austen,Pride and Prejudice,4,279.0,NaN,TRUE
4,goodreads_5,Stephenie Meyer,Stephenie Meyer,Twilight,3,498.0,NaN,None
...,...,...,...,...,...,...,...,...
54295,goodreads_54296,Howard Megdal,Howard Megdal,Taking the Field: A Fan's Quest to Run the Tea...,3,256.0,NaN,None
54296,goodreads_54297,Howard Megdal,Howard Megdal,"The Baseball Talmud: Koufax, Greenberg, and th...",3,256.0,NaN,None
54297,goodreads_54298,Howard Megdal,Howard Megdal,"Wilpon's Folly - The Story of a Man, His Fortu...",3,NaN,NaN,None
54298,goodreads_54299,Mimi Baird/Eve Claxton/,Mimi Baird,He Wanted the Moon: The Madness and Medical Ge...,3,272.0,NaN,TRUE


In [311]:
df=pd.merge(df_recommendation, df_goodreadsMT,  left_on=['title'], right_on = ['title'])

df

,id_x,title,author_x,year,publisher,id_y,Allauthors,author_y,rating,pages,genres,Mul
0,recommendation_6,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,goodreads_927,Amy Tan,Amy Tan,3,416.0,NaN,None
1,recommendation_6,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,goodreads_39164,Amy Tan,Amy Tan,3,415.0,NaN,None
2,recommendation_1430,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,goodreads_927,Amy Tan,Amy Tan,3,416.0,NaN,None
3,recommendation_1430,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,goodreads_39164,Amy Tan,Amy Tan,3,415.0,NaN,None
4,recommendation_27681,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,goodreads_927,Amy Tan,Amy Tan,3,416.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...
32820,recommendation_270885,Radiance,Anne Maybury,1980,Ace Books,goodreads_21680,Grace Draven,Grace Draven,4,297.0,NaN,None
32821,recommendation_270885,Radiance,Anne Maybury,1980,Ace Books,goodreads_46350,Catherynne M. Valente,Catherynne M. Valente,3,429.0,NaN,None
32822,recommendation_270993,The Broken Sword,Northam Anderson,1981,Del Rey Books,goodreads_49628,Poul Anderson,Poul Anderson,3,274.0,NaN,None
32823,recommendation_271207,Pathfinder,James Fenimore Cooper,1982,Signet Book,goodreads_24738,Orson Scott Card,Orson Scott Card,3,657.0,NaN,None


In [312]:
df.drop_duplicates(subset=['id_x'],inplace=True,keep='first')
df.drop_duplicates(subset=['id_y'],inplace=True,keep='first')
df["gold"] =  np.where(df['author_x'] == df['author_y'],True,False)
df.drop(df[df['gold'] == True].index,inplace=True)
df=df.drop(df[df['Mul'] != "TRUE"].index)
df.drop(['title','pages','year','publisher','author_y','rating','genres','Mul'],1,inplace=True)
df

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,id_x,author_x,id_y,Allauthors,gold
338,recommendation_76,Niccolo Machiavelli,goodreads_419,Niccolò Machiavelli/Rufus Goodwin/Benjamin Mar...,False
498,recommendation_134,Gabriel Garcia Marquez,goodreads_37,Gabriel García Márquez/Gregory Rabassa/,False
961,recommendation_273,ROALD DAHL,goodreads_288,Roald Dahl/Quentin Blake/,False
1092,recommendation_311,Mary Barnard,goodreads_15339,Sappho/Mary Barnard/Dudley Fitts/,False
1097,recommendation_317,Isabelle Holland,goodreads_25768,Lee Carroll/Lee Carroll/,False
...,...,...,...,...,...
32806,recommendation_270268,Elizabeth Gibson,goodreads_8543,Plutarch/Arthur Hugh Clough/John Dryden/,False
32808,recommendation_270363,Doris Longacre,goodreads_34912,Doris Janzen Longacre/Mary Emma Showalter Eby/...,False
32810,recommendation_270498,ROLF GATES,goodreads_17558,Rolf Gates/Katrina Kenison/,False
32811,recommendation_270615,Jody Carr,goodreads_43445,Louise Bagshawe/Marie McCarthy/,False


In [314]:
df=df.sort_values(by=['author_x'], ascending=False)
df.reset_index( inplace=True)
df.drop('index',1,inplace=True)
df

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,id_x,author_x,id_y,Allauthors,gold
0,recommendation_55650,ZILPHA KEATLEY SNYDER,goodreads_44716,Zilpha Keatley Snyder/Alton Raible/,False
1,recommendation_18817,Yukio Pseud Mishima,goodreads_11051,Yukio Mishima/Meredith Weatherby/,False
2,recommendation_43755,Yoko Kawashawa Watkins,goodreads_26788,Yoko Kawashima Watkins/Jean Fritz/,False
3,recommendation_57663,Yashar Kemal,goodreads_29282,Yaşar Kemal/Edouard Roditi/,False
4,recommendation_41284,Yan Mo,goodreads_39681,Mo Yan/Howard Goldblatt/,False
...,...,...,...,...,...
789,recommendation_17421,A. N. Roquelaure,goodreads_15926,A.N. Roquelaure/Anne Rice/,False
790,recommendation_124861,A. N. Roquelaure,goodreads_7788,A.N. Roquelaure/Anne Rice/,False
791,recommendation_77830,A. N. Roquelaure,goodreads_6265,A.N. Roquelaure/Anne Rice/,False
792,recommendation_209016,A. Jarry,goodreads_45370,Alfred Jarry/Barbara Wright/Franciszka Themerson/,False


In [315]:
df.loc[:500].to_csv('goodreads_recommendation_H.csv')

# After that some of the data were evaluated by hand and then read again

In [320]:
Gold = pd.read_csv('goodreads_recommendation_H.csv')
Gold.drop('Unnamed: 0',1,inplace=True)
gs_goodreads_recommendation=Gold.drop(['author_x','Allauthors'],1)
gs_goodreads_recommendation.drop(gs_goodreads_recommendation[gs_goodreads_recommendation['gold'] == True].index)
# gs_goodreads_recommendation

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,id_x,id_y,gold
5,recommendation_170779,goodreads_50649,False
6,recommendation_22593,goodreads_23342,False
7,recommendation_176534,goodreads_30410,False
8,recommendation_244960,goodreads_1420,False
9,recommendation_214820,goodreads_4977,False
...,...,...,...
458,recommendation_218668,goodreads_45966,False
489,recommendation_40576,goodreads_49155,False
490,recommendation_7026,goodreads_53617,False
498,recommendation_78424,goodreads_2030,False


# We noticed that  the ratio is about 45:55 between missmatch and matches. So, We will add more Matches values

In [327]:
df=pd.merge(df_recommendation, df_goodreadsMT,  left_on=['title','author'], right_on = ['title','author'])
df.drop_duplicates(subset=['id_x'],inplace=True,keep='last')
df.drop_duplicates(subset=['id_y'],inplace=True,keep='last')
df=df.drop(df[df['Mul'] == "TRUE"].index)
df=df.sort_values(by=['author'], ascending=True)
df.reset_index( inplace=True)
df.drop(['index','title','pages','year','publisher','author','rating','genres','Mul','Allauthors'],1,inplace=True)
df["gold"]=True
df

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


,id_x,id_y,gold
0,recommendation_59207,goodreads_48902,True
1,recommendation_15232,goodreads_14116,True
2,recommendation_220953,goodreads_13553,True
3,recommendation_22989,goodreads_32267,True
4,recommendation_188038,goodreads_49683,True
...,...,...,...
5086,recommendation_34282,goodreads_53296,True
5087,recommendation_240020,goodreads_14662,True
5088,recommendation_51994,goodreads_25440,True
5089,recommendation_228631,goodreads_158,True


In [332]:
gs_goodreads_recommendation_final=gs_goodreads_recommendation.append(df.loc[:256])
gs_goodreads_recommendation_final.reset_index( inplace=True)
gs_goodreads_recommendation_final.drop(['index'],1,inplace=True)
gs_goodreads_recommendation_final.drop(gs_goodreads_recommendation_final[gs_goodreads_recommendation_final['gold'] == True].index)
gs_goodreads_recommendation_final

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,id_x,id_y,gold
0,recommendation_55650,goodreads_44716,True
1,recommendation_18817,goodreads_11051,True
2,recommendation_43755,goodreads_26788,True
3,recommendation_57663,goodreads_29282,True
4,recommendation_41284,goodreads_39681,True
...,...,...,...
753,recommendation_26681,goodreads_1210,True
754,recommendation_18005,goodreads_4590,True
755,recommendation_196740,goodreads_14492,True
756,recommendation_82693,goodreads_50681,True


### Finally We have our gold standard

In [333]:
gs_goodreads_recommendation_final.to_csv('./gs/gs_goodreads_recommendation.csv')